## Part 1: Text Processing

In this part we are importing the movie reviews that we will be working with for this assignment. We are also importing all the modules that will be used in the rest of the code.
We then proceed to handle the files, separate them in two lists, one in which we store the contents of the file, and one where we assign a value of -1 or 1 depending on whether the review was negative or positive, respectively.
We also do a very basic text processing of the reviews, and we also split them into tokens, which will be saved in a list that will later be turned to a set to avoid duplicates.

In [125]:
!wget -N http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
import numpy as np
from numpy.linalg import norm
import random

--2021-04-21 23:34:43--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘review_polarity.tar.gz’ not modified on server. Omitting download.



In [126]:
files = !tar xvzf review_polarity.tar.gz
docs, labels = [], []
files = list(files)
for file in files[:1000]:
     with open(file) as f:
         lines = f.read()
         docs.append(lines)
     labels.append(-1)
for file in files[1000:2000]:
     with open(file) as f:
         lines = f.readline()
         docs.append(lines)
     labels.append(1)

In [127]:
#TEXT PROCESSING
newdocs = []
vocab = []

for doc in docs:
  doc = doc.strip()
  doc = doc.replace('\n', '')
  newdocs.append(doc)
  doc = doc.split()
  vocab.extend(doc)

vocab = sorted(set(vocab))

In [128]:
featuredict = dict()
for i, word in enumerate(vocab):
  featuredict[word] = i

## Part 2: Feature extraction


This is the part that we basically do what the vectorizer would have done and create lists that we will use as our vectors. We then turn the "vectors" into numpy arrays and then assign them all to a list.

In [129]:
allvectors = []
for doc in docs:
  vector = []
  for word in vocab:
    if word in doc:
      vector.append(1)
    else:
      vector.append(0)
  vector = np.array(vector)
  allvectors.append(vector)

## Part 3: Learning framework 


For the convenience of having both training and testing data, we split our dataset here (80/20).

This is the most important part that holds our class "Model" that simulates what sklearn would have done. 

We initialize the class with default values for the learning rate and the regulizer dampening factor and also a hyperplane (omega) that is set to None. 

By calling the fit method, we follow all the required mathematical computations for the Gradient Descent and the Hinge Loss, and we find the our best hyperplane. By calling the method score, we also call the method predict which will give us the predicted results by the model, that is -1 or 1 for each of the reviewed documents and it will eventually yield the accuracy of our model. The methods plot and weights, give us a plot and chart of all the highest weighting words respectively. 

Many of the words given by the weights method appear to be letters, which I attribute to the simple tokenization process that proceeded.

In [130]:
#Splitting the data
X_train = allvectors[:800] + allvectors[1000:1800]
X_test = allvectors[800:1000] + allvectors[1800:2000]
y_train = labels[:800] + labels[1000:1800]
y_test = labels[800:1000] + labels[1800:2000]

In [135]:
class Model:
  def __init__(self, learning_rate = 0.001, regulizer_dampening = 0.001):
    self.omega = None
    self.learning_rate = learning_rate #gamma
    self.regulizer_dampening = regulizer_dampening #lambda
  
  def fit(self, X, y):
    X = np.c_[np.ones((len(X),1)),X]
    self.omega = np.random.normal(0, 1, size=len(X[0]))
    omega_norm = norm(self.omega)**2
    sum = 0
    for i, array in enumerate(X):
      sum += max(0,1 - y[i] * np.vdot(self.omega, array))
    hingeloss = (self.regulizer_dampening/2)* omega_norm + sum
    
    #GRADIENT
    sum2 = np.zeros(len(X[0]))
    for i, array in enumerate(X):
      if y[i] * np.vdot(self.omega, array) < 1:
        sum2 += -(y[i] * array)
    gradient = self.regulizer_dampening * self.omega + sum2     
    self.omega = self.omega - self.learning_rate * gradient
    omega_norm = norm(self.omega)**2
    sum = 0
    for i, array in enumerate(X):
     sum += max(0,1 - y[i] * np.vdot(self.omega, array))
    hingeloss = (self.regulizer_dampening/2)* omega_norm + sum

    #HINGE LOSS
    besthingeloss = hingeloss
    counter = 0
    for i in range(1000):
      sum2 = np.zeros(len(X[0]))
      for i, array in enumerate(X):
        if y[i] * np.vdot(self.omega, array) < 1:
          sum2 += -(y[i] * array)
        gradient = self.regulizer_dampening * self.omega + sum2 #GRADIENT DESCENT
        self.omega = self.omega - self.learning_rate * gradient #OMEGA
      sum = 0
      for i, array in enumerate(X):
        sum += max(0,1 - y[i] * np.vdot(self.omega, array))
      hingeloss = (self.regulizer_dampening/2)* norm(self.omega) + sum  
      if hingeloss > besthingeloss - 0.001:
        counter +=1 
      if hingeloss > besthingeloss - 0.001 and counter == 5:
        hingeloss = besthingeloss
        break
      if hingeloss < besthingeloss:
        besthingeloss = hingeloss

  def predict(self, X, y):
    X = np.c_[np.ones((len(X),1)),X]
    predicted = []
    for vector in X:
      result = np.matmul(self.omega, vector)
      signfunction = np.sign(result)
      predicted.append(signfunction)
    return predicted

  def score(self, X, y):
    predicted = self.predict(X,y)
    counter = 0
    for i, element in enumerate(y):
      if element == predicted[i].item(0):
        counter += 1
    return counter/len(y)*100
  
  def plot(self):
    import matplotlib.pyplot as plt
    x = np.arange(0, len(self.omega.T), 1)
    plt.figure(figsize=(20, 3))
    plt.plot(x[1:], self.omega[1:])
    plt.xlabel("Value")
    plt.ylabel("Weights")
    plt.show()

  def weights(self, X_raw, vocab):
    idx = np.argsort(np.abs(self.omega[1:]))
    print("                Word   Weight  Occurences")
    for i in idx[-20:]:  
      print("%20s   %.3f\t%i " % (vocab[i], self.omega[i+1], np.sum([vocab[i] in d for d in X_raw])))

In [136]:
#model = Model()
#model.fit(X_train, y_train)
#model.score(X_train, y_train)
#model.weights(docs, vocab)

                Word   Weight  Occurences
                more   -95.070	781 
                this   -95.533	1079 
                   p   99.781	1703 
                   d   100.010	1888 
                   .   102.990	1905 
                  er   106.225	1711 
                   m   106.797	1908 
                   l   107.111	1921 
                  he   109.114	1742 
                   h   111.926	1949 
                   b   114.700	1715 
                   r   116.271	1962 
                   n   116.559	1964 
                   s   118.587	1979 
                   t   119.846	1984 
                   a   120.223	1984 
                   o   120.262	1983 
                   i   121.449	1988 
                   e   122.058	1990 
                   c   131.487	1844 


## Part 4: Exploring hyperparameters


For the final part of the code, we iterate through 10 different combinations of hyperparameters and we check to see which one performed the best. We then take these specific ones and use them to apply our model to the test data that we have reserved.

In [137]:
best_hyperparameters = None
grid = {'learning_rate': np.exp(np.linspace(np.log(0.0001), np.log(3), 10)),
        'reguliser_dampening': np.exp(np.linspace(np.log(0.0001), np.log(3), 10))}

print("Learning rate:\tReg.dampening:\tTraining set accuracy:")

for i in range(10):
  lr = grid['learning_rate'][random.randint(0,9)]
  rd = grid['reguliser_dampening'][random.randint(0,9)]
  model = Model(learning_rate = lr, regulizer_dampening = rd )
  model.fit(X_train, y_train)
  training_accuracy = model.score(X_train, y_train)
  if best_hyperparameters is None or best_hyperparameters[2] < training_accuracy:
    best_hyperparameters = (lr, rd, training_accuracy)
  print("%.5f\t\t%.5f\t\t%.1f%%" % (lr, rd, training_accuracy))

best_learning_rate = best_hyperparameters[0]
best_reguliser_dampening = best_hyperparameters[1]
print("Best parameters: %.5f, %.5f" % (best_learning_rate, best_reguliser_dampening))

Learning rate:	Reg.dampening:	Training set accuracy:
0.00311		0.00031		99.5%
0.00031		0.00099		100.0%
0.00311		0.00311		99.8%
0.00099		0.09655		80.2%
0.30353		0.00311		89.8%
0.95425		0.00311		97.4%
0.00977		0.00031		100.0%
0.09655		0.95425		60.8%
0.09655		3.00000		50.1%
0.00099		0.03071		99.5%
Best parameters: 0.00031, 0.00099


In [138]:
model = Model(learning_rate = best_learning_rate, regulizer_dampening = best_reguliser_dampening)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print("Test Accuracy:", acc)

Test Accuracy: 99.75


# Qualitative Analysis

With the convergence of the hinge loss and the creation of the hyperplane, we obtained high accuracy scores for both the training and the test set.
By looking at the weights for the words with the highest 'voting' values, it is easy to understand that something went wrong with the tokenization of the documents most probably. It is not very sound to encounter single letters that have such positive weights. However, there are still 2 full words, "more" and "this", which are associated with around -95. This actually makes sense for "more", since if it were to be used in a movie review it would call for "more" of something, rendering the movie less than what it should have been.

The hyperparameters that were found to be the best (for one instance) were 0.00031 and 0.00099, which can denote that a smaller but steadier learning rate and regulizer dampening are more suitable for our model. Having the initial learning rate of 0.1 resulted in a hinge loss 100x bigger than the first hinge loss produced, and lowering the learning rate gave a better balanced outcome.

In terms of design, the choice of a class for implementing the model was ideal, as it can withhold information and it is easier to access class methods. In terms of data structures, I made use of arrays, lists and dictionaries, which facilitated different functions each. Arrays were extremely helpful for indexing and accessing specific elements or rows/columns, as well as for mathematical operations which were heavily used througout the assignment.
One of the choices I made that I think I should comment on is the choice of a for loop for 1000 iterations for the search of the best hinge loss instead of a while loop, which was based on the idea of a more efficient code that takes less time in the case that the best hinge loss is not found.

# Comments

In general, the implementation of the whole assignment was a tough challenge for people with minimal math knowledge, and trying to code the formulas without really understanding the theory behind them proved to work but it definitely left us greatly confused. However it was indeed educational since we got to understand what the backbone of a Machine Learning model looks like and provided us with deep insight into its workings. Part 3 was definitely the one that required the most time out of all the assignment and also was the hardest to implement.
